<a href="https://colab.research.google.com/github/manuuC3368/CodeLab_BI/blob/main/CreateDataSet_with_APIservices_ManCas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.4 MB/s eta 0:00:00


In [2]:
import sqlite3
import pandas as pd
import requests
import random
import uuid
from faker import Faker
import numpy as np
import shutil
from google.colab import drive

In [3]:
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Inicializar Faker
fake = Faker()

# Crear la conexión a la base de datos SQLite3
conn = sqlite3.connect('financial_data.db')

# Función para ejecutar consultas SQL
def execute_query(query, conn):
    with conn:
        conn.execute(query)

# Crear las tablas en SQLite3
execute_query('''CREATE TABLE IF NOT EXISTS customers (
                 customer_id TEXT PRIMARY KEY,
                 name TEXT,
                 address TEXT,
                 phone_number TEXT,
                 email TEXT);''', conn)

execute_query('''CREATE TABLE IF NOT EXISTS branches (
                 branch_id TEXT PRIMARY KEY,
                 branch_location TEXT,
                 manager_name TEXT,
                 contact_number TEXT);''', conn)

execute_query('''CREATE TABLE IF NOT EXISTS transaction_types (
                 transaction_type TEXT PRIMARY KEY,
                 description TEXT);''', conn)

execute_query('''CREATE TABLE IF NOT EXISTS transactions (
                 transaction_id TEXT PRIMARY KEY,
                 customer_id TEXT,
                 transaction_date TEXT,
                 transaction_amount REAL,
                 transaction_location TEXT,
                 transaction_type TEXT,
                 fraudulent INTEGER,
                 branch_id TEXT,
                 FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
                 FOREIGN KEY(transaction_type) REFERENCES transaction_types(transaction_type),
                 FOREIGN KEY(branch_id) REFERENCES branches(branch_id));''', conn)

In [5]:
# Función para obtener datos de clientes desde randomuser API
def get_random_users(num_users=10):
    url = f"https://randomuser.me/api/?results={num_users}&nat=us"
    response = requests.get(url)
    if response.status_code == 200:
        users = response.json()['results']
        return users
    else:
        print("Error fetching data from randomuser.me")
        return []

# Crear la tabla de clientes y guardarla en SQLite3
def create_customers_table(num_customers=10):
    users = get_random_users(num_customers)
    customers_data = {
        "customer_id": [str(uuid.uuid4()) for _ in range(num_customers)],
        "name": [f"{user['name']['first']} {user['name']['last']}" for user in users],
        "address": [f"{user['location']['street']['number']} {user['location']['street']['name']}, {user['location']['city']}, {user['location']['state']}, {user['location']['postcode']}" for user in users],
        "phone_number": [user['phone'] for user in users],
        "email": [user['email'] for user in users]
    }
    customers_df = pd.DataFrame(customers_data)
    customers_df.to_sql('customers', conn, if_exists='replace', index=False)
    return customers_df

# Crear la tabla de clientes
customers_df = create_customers_table(num_customers=100)